# Second Part

## Глава 3

### Словари и множества

In [4]:
# Хэш-таблицы - словари, dict, {} - широко используются
#     как в пользовательских кодах, так и в реализации Python.

# Все встроенные функции хранятся в словаре __builtins__.__dict__.

# Кроме словарей, в данной главе будут также рассмотрены множества:
#     структуры данных, тоже реализованные с помощью хэш-таблиц.


# Краткое содержание главы:

#     - Часто используемые методы словаря;

#     - Специальная обработка отсутствия ключа;

#     - Различные вариации типа dict в стандартной библиотеке;

#     - Типы set и frozenset;

#     - Как работают хэш-таблицы;

#     - Следствия механизма работы хэш-таблиц.

### Общие типы отображений

In [6]:
# Модуль collections.abc содержит абстрактные базовые классы
#     Mapping и MutapbleMapping, формализирующие интерфейсы dict.

In [10]:
from collections import abc
my_dict = {}
isinstance(my_dict, abc.Mapping)

True

##### Ключи словаря должны быть хэшируемыми!

In [12]:
# Объект называется хэшируемым, если имеет хэш-значение,
#     которое не изменеся на протяжении времени его жизни
#         (у него должен быть метод __hash__()),
#             и допускает сравнение с другими объектами
#                 (у него должен быть метод __eq__()).

# У равных объектов должны быть равны хэш-значеничя.

In [13]:
# Все атомарные неизменяемые типы (str, bytes, числовые типы)
#     являются хэшируемыми.
# Объект типа frozenset всегда хэшируемый,
#     т.к. все его элементы хэшируемые по определению.

# Объект типа tuple является хэшируемым только тогда,
#     когда хэшируемы все его элементы.

In [14]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [17]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [20]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

-4118419923444501110

In [22]:
# Любой пользовательский тип является хэшируемым по определению,
#     т.к. его хэш-значение равно id(), и id() объекта уникален.

# Если объект реализует метод __eq__, то он будет хэшируемым,
#     только если все атрибуты неизменяемые.

In [25]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

### Словарное включение

In [27]:
# Dictcomp строит объект dict, 
#     порождая пары key:value из итерируемого объекта.

In [31]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan')
]

country_code = {country: code for code, country in DIAL_CODES}
country_code

{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [32]:
{code: country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 7: 'RUSSIA', 55: 'BRAZIL', 62: 'INDONESIA'}

### Обзор наиболее употребительных методов отображений

In [34]:
# Взглянем на метод .update(m, [**kargs]):

# Сначала он проверяет, есть ли у m метод keys,
#     если да, то он предпалагается как отображение.

# В противном случае update производит обход m
#     в предположении, что элементами яявляются пары (key, value).

# Это пример динамическое типизации (duck typing).

#### Обработка отсутствия ключей с помощью setdefault

In [36]:
# Доступ к словарю dict[key] возбуждает исключение,
#     если ключ отсутствует.

# Конструкция dict.get(k, default) позволяет 
#     обрабатывать значение по умолчанию вместо KeyError.

# Тем не менее, и данный способ может быть неоптимальным:

In [41]:
import os
os.listdir()

['.ipynb_checkpoints',
 'Fluent Python Chapter 2.ipynb',
 'Fluent Python Chapter 3.ipynb',
 'zen.txt']

In [44]:
import sys
import re

WORD_RE = re.compile('\w+')

index = {}

with open('zen.txt', encoding='cp1252') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            # не стоит так писать:
            occurences = index.get(word, []) # список вхождений или []
            occurences.append(location) # добавляем новое вхождение
            index[word] = occurences # помещаем список в словарь

for word in sorted(index, key=str.upper): # передаем key ссылку  на метод
    print(word, index[word])

a [(16, 48), (17, 48), (18, 53)]
Although [(9, 1), (14, 1), (16, 1)]
ambiguity [(12, 16)]
and [(13, 21)]
are [(19, 12)]
aren [(8, 15)]
at [(14, 38)]
bad [(17, 50)]
be [(13, 14), (14, 27), (18, 50)]
beats [(9, 23)]
Beautiful [(1, 1)]
better [(1, 14), (2, 13), (3, 11), (4, 12), (5, 9), (6, 11), (15, 8), (16, 25)]
break [(8, 40)]
cases [(8, 9)]
complex [(3, 23)]
Complex [(4, 1)]
complicated [(4, 24)]
counts [(7, 13)]
dense [(6, 23)]
do [(13, 60), (19, 45)]
Dutch [(14, 61)]
easy [(18, 26)]
enough [(8, 30)]
Errors [(10, 1)]
explain [(17, 34), (18, 34)]
Explicit [(2, 1)]
explicitly [(11, 8)]
face [(12, 8)]
first [(14, 41)]
Flat [(5, 1)]
good [(18, 55)]
great [(19, 28)]
guess [(12, 52)]
hard [(17, 26)]
honking [(19, 20)]
idea [(17, 54), (18, 60), (19, 34)]
If [(17, 1), (18, 1)]
implementation [(17, 8), (18, 8)]
implicit [(2, 25)]
In [(12, 1)]
is [(1, 11), (2, 10), (3, 8), (4, 9), (5, 6), (6, 8), (15, 5), (16, 16), (17, 23), (18, 23)]
it [(13, 63), (17, 43), (18, 43)]
let [(19, 39)]
may [(14, 

In [45]:
WORD_RE = re.compile('\w+')

index = {}

with open('zen.txt', encoding='cp1252') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            
            index.setdefault(word, []).append(location)

for word in sorted(index, key=str.upper): # передаем key ссылку  на метод
    print(word, index[word])

a [(16, 48), (17, 48), (18, 53)]
Although [(9, 1), (14, 1), (16, 1)]
ambiguity [(12, 16)]
and [(13, 21)]
are [(19, 12)]
aren [(8, 15)]
at [(14, 38)]
bad [(17, 50)]
be [(13, 14), (14, 27), (18, 50)]
beats [(9, 23)]
Beautiful [(1, 1)]
better [(1, 14), (2, 13), (3, 11), (4, 12), (5, 9), (6, 11), (15, 8), (16, 25)]
break [(8, 40)]
cases [(8, 9)]
complex [(3, 23)]
Complex [(4, 1)]
complicated [(4, 24)]
counts [(7, 13)]
dense [(6, 23)]
do [(13, 60), (19, 45)]
Dutch [(14, 61)]
easy [(18, 26)]
enough [(8, 30)]
Errors [(10, 1)]
explain [(17, 34), (18, 34)]
Explicit [(2, 1)]
explicitly [(11, 8)]
face [(12, 8)]
first [(14, 41)]
Flat [(5, 1)]
good [(18, 55)]
great [(19, 28)]
guess [(12, 52)]
hard [(17, 26)]
honking [(19, 20)]
idea [(17, 54), (18, 60), (19, 34)]
If [(17, 1), (18, 1)]
implementation [(17, 8), (18, 8)]
implicit [(2, 25)]
In [(12, 1)]
is [(1, 11), (2, 10), (3, 8), (4, 9), (5, 6), (6, 8), (15, 5), (16, 16), (17, 23), (18, 23)]
it [(13, 63), (17, 43), (18, 43)]
let [(19, 39)]
may [(14, 

### Отображения с гибким поиском по ключу

In [47]:
# Удобно, чтобы отображение возвращало специальное значение,
#     если искомый ключ отсутствует. 

# Для реализации можно как использовать класс defaultdict,
#     так и создать подкласс отображения с методом __missing__.

#### Defaultdict: еще один подход к обработке отсутствия ключа

In [49]:
# Объект collections.defaultdict элегантно решает предыдущую задачу.

# Он сконфигурирован так, что по запросу возвращает элементы,
#     когда искомый ключ отсутствует:

#         - При конструировании объекта defaultdict задается вызываемый объект;
         
#         - Объект порождает значение по умолчанию;

#         - Это происходит каждый раз, 
#               когда методу __getitem__ передается ключ,
#                   отсутствующий в словаре.

# Например, dd = defaultdict(list).

# Если ключ 'new-key' отсутствует в dd, 
#     при вычислении dd['new-key'] выполняются следующие действия:

#         1. Вызывается list() для создания нового списка;
#         2. Список вставляется в dd в качестве значения ключа 'new-key';
#         3. Возвращается ссылка на список.

In [50]:
import os
os.listdir()

['.ipynb_checkpoints',
 'Fluent Python Chapter 2.ipynb',
 'Fluent Python Chapter 3.ipynb',
 'zen.txt']

In [52]:
import sys
import re
import collections

WORD_RE = re.compile('\w+')

index = collections.defaultdict(list) # создаем defaultdict с конструктором list
with open('zen.txt', encoding='cp1252') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            index[word].append(location) # если word еще нет в index,
                            # то вызывается функция default_factory,
                            # порождающее отсутствующее значение;
                            # значение присваивается и возвращается,
                            # так что append всегда завершается успешно

for word in sorted(index, key=str.upper):
    print(word, index[word])

a [(16, 48), (17, 48), (18, 53)]
Although [(9, 1), (14, 1), (16, 1)]
ambiguity [(12, 16)]
and [(13, 21)]
are [(19, 12)]
aren [(8, 15)]
at [(14, 38)]
bad [(17, 50)]
be [(13, 14), (14, 27), (18, 50)]
beats [(9, 23)]
Beautiful [(1, 1)]
better [(1, 14), (2, 13), (3, 11), (4, 12), (5, 9), (6, 11), (15, 8), (16, 25)]
break [(8, 40)]
cases [(8, 9)]
complex [(3, 23)]
Complex [(4, 1)]
complicated [(4, 24)]
counts [(7, 13)]
dense [(6, 23)]
do [(13, 60), (19, 45)]
Dutch [(14, 61)]
easy [(18, 26)]
enough [(8, 30)]
Errors [(10, 1)]
explain [(17, 34), (18, 34)]
Explicit [(2, 1)]
explicitly [(11, 8)]
face [(12, 8)]
first [(14, 41)]
Flat [(5, 1)]
good [(18, 55)]
great [(19, 28)]
guess [(12, 52)]
hard [(17, 26)]
honking [(19, 20)]
idea [(17, 54), (18, 60), (19, 34)]
If [(17, 1), (18, 1)]
implementation [(17, 8), (18, 8)]
implicit [(2, 25)]
In [(12, 1)]
is [(1, 11), (2, 10), (3, 8), (4, 9), (5, 6), (6, 8), (15, 5), (16, 16), (17, 23), (18, 23)]
it [(13, 63), (17, 43), (18, 43)]
let [(19, 39)]
may [(14, 

In [53]:
# Атрибут default_factory объекта defaultdict вызывается для того,
#     чтобы предоставить значение по умолчанию 
#         при обращении к методу __getitem__;

# Если dd - объект класса defaultdict и k - отсутствующий ключ,
#     при вычислении dd[k] происходит обращение к default_factory
#         для создания значения по умолчанию;

# Вызов же dd.get(k) все равно возвращает None.

#### Метод __missing__

In [55]:
# Метод __missing__ лежит в основе мехинизма 
#     обработки отсутствия ключей в отображениях;

# Если создать подкласс dict и реализовать в нем метод __missing_,
#     стандартный метод dict.__getitem__ будет обращаться к нему
#         всякий раз, как не найдет ключ (вместо возбуждения KeyError).

In [56]:
# Метод __missing__ вызывается только из метода __getitem__ 
#     (то есть при выполнении оператора d[k]);

# Наличие __missing__ не влияет на поведение других методов,
#     производящих поиск по ключу, например get или __contains__ (оператор in);

# Поэтому атрибут default_factory объекта defaultdict 
#     работает только с __getitem___.

In [57]:
# Допустим, нам требуеутся отображение,
#     в котором ключ перед поиском преобразуется в тип str:

In [59]:
class StrKeyDict0(dict): # StrKeyDict0 наследует dict
    
    def __missing__(self, key):
        if isinstance(key, str): # проверяем, принадлежит ли key типу str
            raise KeyError(key) # если да и отсутсвует, то возбуждаем исключ-е
        return self[str(key)] # преобразуем key в str и ищем
    
    def get(self, key, default=None):
        try:
            return self[key] # get делегирует работу методу __getitem__
        except KeyError:
            return default # метод __missing__ уже выдал ошибку
    
    def __contains__(self, key): # ищем сначала key, потом - str(key)
        return key in self.keys() or str(key) in self.keys()
    
# без проверки isinstance(key, str) в случае, 
#     если ключ str(k) не существует, возникает бесконечная рекурсия

# метод __contains__ отвечает за проверку через оператор in,
#     но не обращается к __missing__ в случае отсутствия ключа;

# в Python 3 поиск вида k in my_dict.keys() эффективен 
#     даже для очень больших отображений, в отличие от Python 2

In [62]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])

In [63]:
d['2']

'two'

In [64]:
d[4]

'four'

In [69]:
try:
    d[1]
except KeyError:
    print(True)

True


In [70]:
d.get('2')

'two'

In [71]:
d.get(4)

'four'

In [73]:
d.get(1, 'N/A')

'N/A'

In [75]:
2 in d

True

In [77]:
'2' in d

True

In [76]:
1 in d

False

### Вариации на тему dict

In [79]:
# Помимо defaultdict в стандартную библиотеку 
#     включены следующие типы отображений:

##### collections.OrderedDict

In [ ]:
# Ключи хранятся в том порядке, в котором вставлялись - порядок предсказуем;

# Метод popitem по умолчанию удаляет и возвращает первый элемент,
#     но если вызывается так: my_odict.popitem(last=True), то последний.

##### collections.ChainMap

In [82]:
# Хранит список отображений, 
#     так что их можно просматривать как единое целое;

# Поиск производится в каждом отображении по порядку и завершается успешно,
#     если ключ найден хотя бы в одном.

In [85]:
from collections import ChainMap
import builtins

pylookup = ChainMap(locals(), globals(), vars(builtins))

##### collections.Counter

In [87]:
# Отображение, в котором с каждым ключом ассоциирован счетчик;

# Класс можно использовать для подсчета хэшируемых объектов (ключей)
#     или в качестве мультимножества - множества, 
#         в которое каждый элемент может входить несколько раз;


# В классе Counter реализованы операторы + и - для объединения серий,
#     а также другие полезные методы, как most_common([n]),
#         возвращающий упорядоченный список кортежей,
#             содержищий n самых часто встречающихся элементов
#                 вместе с их счетчиками.

In [90]:
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [91]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'b': 2, 'c': 1, 'd': 1, 'r': 2, 'z': 3})

In [92]:
ct.most_common(2)

[('a', 10), ('z', 3)]

##### collections.UserDict

In [94]:
# Реализацияя на чистом Python отображения,
#     работающего как стандартный словарь dict;

# UserDict редназначен для наследования.

### Создание подкласса UserDict

In [97]:
# Почти всегда проще создать новый тип отображения
#     путем расширения UserDict, а не dict;

# Преимущество заключается в том,
#     что реализация dict вынуждает нас переопределять те методы,
#         которые мы могли бы без проблем унаследовать от UserDict;

# UserDict не наследует dict, 
#     а хранит внутри себя экземпляр dict в атрибуте data,
#         где и находятся сами элементы.

# Это позволяет избегать нежелательных рекурсий
#     и упрощает код __contains__.

In [ ]:
import collections

class StrKeyDict(collections.UserDict): # StrKeyDict расширяет UserDict
    
    def __missing__(self, key): # метод __missing__ остается прежним
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data # метод __contains__ реализован проще
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item # преобразует любой ключ в str

In [ ]:
# Благодаря UserDict класс StrKeyDict получился короче StrKeyDict0,
#     но умеет при этом больше: он хранит все ключи в виде str,
#         обходя неприятные сюрпризы с нестроковыми ключами.

In [98]:
# Поскольку UserDict - подкласс MutableMapping, 
#     остальные использующиеся методы отображения
#         наследуются от UserDict, MutableMapping или Mapping;

# В двух последних также есь полезные методы:

##### MutableMapping.update

In [100]:
# Этот метод вызывается как напрямую, 
#     так и методом __init__ для инициализации экземпляра отображениями,
#         и итерируемыми объектами с парами (key, value),
#             и именованными аргументами;

# Поскольку для добавления элементов 
#     в нем используется конструкция self(key) = value,
#         то в конечном итоге будет вызвана реализация __setitem__.

##### Mapping.get

In [ ]:
# Метод Mapping.get, унаследованный StrKeyDict,
#     реализован так же, как метод get в StrKeyDict0 
#         (последний реализован нами самостоятельно).

### Неизменяемые отображения

In [103]:
# Иноогда нужно гарантировать, 
#     что пользователь не может по ошибке модифицировать отображение.

# На помощь приходит класс-обертка MappingProxyType:

In [104]:
from types import MappingProxyType

d = {1: 'A'}

d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [105]:
d_proxy[1]

'A'

In [106]:
d_proxy[2]

KeyError: 2

In [107]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [108]:
d[2] = 'B'

In [109]:
d_proxy # представление d_proxy динамическое

mappingproxy({1: 'A', 2: 'B'})

In [110]:
d_proxy[2]

'B'

### Теория множеств

##### Множество - набор уникальных объектов

In [115]:
l = ['spam', 'spam', 'eggs', 'spam']
list(set(l))

['spam', 'eggs']

##### Элементы множества должны быть хэшируемыми

In [117]:
# Сам тип set хэшируемым не является,
#     но тип frozenset хэшируемый,
#         поэтому объекты типа frozenset могут быть элементами множества.

In [118]:
# Множества предоставляют набор операций:

#     - a | b - оъединение множеств;

#     - a & b - пересечение множеств;

#     - a - b - разность множеств.

# Использование множественных операций помогает  
#     как уменьшить время работы и объем программы,
#         так и облегчить восприятие и осмысление кода.

In [119]:
# Допустим, у нас есть большой набор почтовых адресов (haystack)
#     и меньший набор адресов (needles);

# Наша задача - подсчитать, 
#     сколько раз элементы needles встречаются в haystack.

# Множества предлагают элегантное решение:

In [ ]:
found = len(set(needles) & set(haystack))

In [121]:
# Без использования пересечений пришлось бы строить цикл:

In [ ]:
found = 0
for n in needles:
    if n in haystack:
        found += 1

##### Быстрая проверка вхождения в set и frozenset обеспечивается механизмом хэш-таблиц

#### Литеральные множества

##### Для создания пустого множества следует использовать set()

In [123]:
# Написав {}, мы объявим пустой словарь вместо множества!

In [137]:
from dis import dis
dis('{1}') # специальный байт-код BUILD_SET выполняет почти всю работу

  1           0 LOAD_CONST               0 (1)
              3 BUILD_SET                1
              6 RETURN_VALUE


In [138]:
dis('set([1])') # вместо BUILD_SET - LOAD_NAME, BUILD_LIST и CALL_FUNCTION

  1           0 LOAD_NAME                0 (set)
              3 LOAD_CONST               0 (1)
              6 BUILD_LIST               1
              9 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             12 RETURN_VALUE


In [135]:
set([1]) == {1}

True

In [139]:
# Для frozenset не существует строковых литералов,
#     так что их приходится создавать с помощью конструктора:

In [141]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

#### Множественное включение

In [146]:
from unicodedata import name # для получения названий символов

{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

#### Операции над множествами

In [148]:
...

Ellipsis

### Под капотом dict и set

In [150]:
# Рассмотрим следующие вопросы:

#     - Насколько эффективны классы dict и set в Python?

#     - Почему они не упорядочены?

#     - Почему не каждый объект Python может быть элементом множества?

#     - Почему нельзя добавлять в множество элементы во время обхода?

#### Экспериментальная демонстрация производительности

In [47]:
...

Ellipsis

#### Хэш-таблицы в словарях

##### Сначала рассмотрим устройство класса dict, а затем распространим идеи на множества

In [3]:
# Хэш-таблица - разреженный массив
#     (массив, в котором имеются незаполненные позиции);

# В хэш-таблице каждому элементу соответстует ячейка,
#     содержащая два поля: ссылку на ключ и ссылку на значение;

# Python стремится оставить не менее трети ячеек пустыми:
#     если хэш-таблица становится заполненной, 
#         она копируется в новый участок памяти;

# Для помещения элемента в хэш-таблицу
#     нужно первым делом вычислить хэш-значение элемента;

# Это делает встроенная функция hash().

##### Хэш-значенияя и равенство

In [4]:
# Встроенная функция hash() работает со встроенными типами напрямую,
#     а для пользовательских типов обращается к методу __hash__;

# Два объекта с одинаковым значением
#     имеют одинаковые хэш-значения - идея алгоритма хэш-таблицы;

In [7]:
assert hash(1.0) == hash(1)

In [9]:
# Хэш-значения эффективны, 
#     когда равномерно распределены по всему пространству индексов;

# Так что хэши похожих, но неодинаковых объектов
#     должны сильно различаться.

In [10]:
# С версии Python 3.3 в хэши объектов str, bytes и datetime
#     добавлена случайная завтравка;

# Ее смысл - защититься от DOS-атак.

##### Алгоритм работы хэш-таблицы

In [13]:
# Для выборки значения с помощью выражения my_dict(search_key)
#     Python обращается к функции hash(search_key),
#         получает хэш-значение search-key;

# Если найденная ячейка пуста, возбуждается исключение KeyError;

# В противном случае в ячейке есть элемент -
#     пара (found_key: found_value), и тогда Python проверяет,
#         верно ли, что search_key == found_key;

# Если да, то элемент найден и возваращется found_value.

In [16]:
# Если же search_key и found_key не совпали,
#     то имеет место коллизия хэширования.

In [15]:
# В случае вставки/изменения элемента 
#     прошлое значение заменяется новым.

#### Практические последствия механизма работы dict

##### Ключи должны быть хэшируемыми объектами

In [19]:
# Объект хэшируемый, если удовлетворяет всем следующим условиям:

#     - Поддерживает функцию hash() 
#           благодаря наличию метода __hash__,
#               возвращающего одно и то же значение 
#                   на протяжении всей жизни объекта;

#     - Поддерживает сравнение на равенство
#           с помощью метода __eq__;

#     - Если выражение a == b равно True, 
#           то выражение hash(a) == hash(b) также должно быть равно True.

##### Пользовательские типы по умолчанию хэшируемые

In [21]:
# Так как их хэш-значение равно уникальному значению функции (id).

In [22]:
# Если в классе реализован метод __eq__,
#     мы должны согласованно реализовать и метод __hash__, 
#         так как необходимо гарантировать,
#             что если a == b равно True,
#                 то и hash(a) == hash(b) равно True;

# Иначе будет нарушен алгоритм хэш-таблицы, 
#     а словари и множества не смогут функционировать.

# Если метод __eq__ зависит от изменяемого состояния,
#     то метод __hash__ должен возбуждать исключение TypeError
#         с сообщением вроде unhashable type: 'MyClass'.

##### У словарей большие накладные расходы в части памяти

In [25]:
# При использовании разреженных таблиц память используеутся неэффективно;

# При обработке большого числа записей 
#     лучше хранить их в списке кортежей или именованных кортежей,
#         а не в списке кортежей в духе JSON,
#             с одним объектом dict на каждую запись.

# Замна словарей кортежами снижает потреблением памяти:

#     - За счет устранения накладных расходов 
#           на хранение хэш-таблицы в каждой записи;

#     - В силу того, что имена полей 
#           вынесены за пределы записей. 

# Оптимизация оправдана в случае работы
#     с действительно большими объемами.

##### Поиск по ключу выполняется очень быстро

In [27]:
# Реализация dict жертвует памятью ради скорости:
#     накладные расходы словаря велики,
#         зато доступ производится быстро независимо от размера словаря.

##### Упорядочение ключей зависит от порядка вставки

In [ ]:
# При возникновении коллизий второй ключ оказывается в позиции,
#     которую не должен был занимать, если бы был вставлен первым;

# Соответственно, в случае коллизий порядок ключей
#     различается в двух словарях d([(k1: v1), (k2: v2)])
#         и d([(k2: v2), (k1: v1)]);

##### При добавлении новых элементов в словарь может измениться порядок существующих ключей

In [39]:
# При добавлении нового элемента интерпретатор может решить,
#     что хэш-таблицу следует перестроить;

# Во время операции перестановки может возникнуть коллизия,
#     из-за чего ключи будут упордочены по-другому;

# Соответственно, при обходе всех ключей словаря
#     и одновременном их изменении могуть быть пропущены элементы;

# Безопаснее работать с копиями словаря благодаря методу copy().

#### Как работают множества

In [41]:
# И set, и frozenset реализованы с помощью хэш-таблиц,
#     в каждой ячейке которых хранится ссылка на элемент
#         (будто ключ словаря, но без значения);

# Элементы множества должны быть хэшируемыми объектами;

# Множества включают в себя значительные накладные расходы в памяти;

# Проверка принадлежности к множеству очень эффективна;

# Упорядочение элементов зависит от порядка вставки;

# Добавление новых элементов в множество
#     может привести к изменению порядка существующих.     

### Резюме

##### Словари - краеугольный камень Python.

In [44]:
# Помимо базового класса dict, 
#     в стандартной библиотеке есть удобные отображения:

#         - collections.defaultdict;

#         - collections.OrderedDict;

#         - collections.ChainMap;

#         - collections.Counter;

# Там же находится класс расширения UserDict.

In [45]:
# Полезные методы большинства отображений -
#     setdefault и update;

# Метод setdefault модифицирует элементы,
#     содержащие изменяемые значения (такие как list),
#         чтобы избежать повторных операций поиска того же ключа;

# Метод update облегчает массовую вставку и перезапись элементов,
#     когда новые элементы берутся из другого отображения,
#         из итерируемого объекта с парами (key: value)
#             или из именованных аргументов;

# Конструкторы отображений также пользуются методом update.

In [ ]:
# В API отображений имеется метод __missing__, позволяющий определить, 
#     что должно происходить в случае отсутствия ключа.

In [46]:
# Модуль collections.abc содержит абстрактные базовые классы 
#     Mapping и MutableMapping для справки и контроля типов;

# Малоизвестный класс MappingProxyType из модуля types
#     позволяет создавать неизменяемые отображения.

In [ ]:
# Реализация хэш-таблицы, лежащей в основе классов dict и set,
#     работает очень быстро;

# А понимание ее логики объясняет,
#     почему элементы кажутся неупорядоенными, 
#         и их порядок может неожиданно измениться;

# Расплачиваемся же мы за быстродействие памятью.